# 08 - Módulos y Paquetes: organizar y reutilizar código

## Objetivos de Aprendizaje

En esta sesión aprenderás:

1. ✅ Definir qué es un módulo y un paquete
2. ✅ Usar `import`, `from` y alias con `as`
3. ✅ Entender el orden de búsqueda y el papel de `sys.path`
4. ✅ Crear módulos propios y reutilizarlos
5. ✅ Crear paquetes con `__init__.py` y submódulos
6. ✅ Diferenciar importaciones absolutas y relativas
7. ✅ Identificar errores comunes al importar y cómo corregirlos
8. ✅ Resolver ejercicios prácticos con módulos y paquetes

---

## Ruta de la sesión (secuencia ideal)

1. ¿Qué es un módulo?
2. Importaciones básicas y alias
3. ¿Dónde busca Python los módulos?
4. Crear y usar módulos propios
5. `__name__` y bloques ejecutables
6. Caché de importación y recarga
7. Paquetes y `__init__.py`
8. Importaciones absolutas vs relativas
9. Errores comunes y buenas prácticas
10. Ejercicios integradores


## 1. ¿Qué es un módulo?

Un **módulo** es un archivo `.py` que contiene variables, funciones y clases.
Cuando haces `import`, Python **ejecuta el archivo una sola vez** y lo guarda en memoria
para reutilizarlo sin volver a ejecutarlo (caché en `sys.modules`).

Ventajas principales:
- Organiza tu código en partes pequeñas y reutilizables.
- Evita duplicar lógica.
- Facilita pruebas y mantenimiento.

⚠️ **Error común**: llamar a tu archivo `random.py`, `json.py` o `math.py`.
Eso “pisará” los módulos estándar y causará errores extraños al importar.


In [ ]:
import math

print(math.pi)
print(math.sqrt(49))
print(math.factorial(5))


## 2. Formas de importar (y cuándo usar cada una)

Hay varias formas de importar:

- `import modulo`: importas todo el módulo y accedes con `modulo.nombre`.
- `import modulo as alias`: útil para nombres largos.
- `from modulo import nombre`: importas solo lo que necesitas.
- `from modulo import nombre as alias`: alias para un nombre específico.

⚠️ **Error común**: usar `from modulo import *`.
Dificulta la lectura, puede sobrescribir nombres y complica el depurado.


In [ ]:
import random as rnd
from statistics import mean, median
from math import pi as PI

numeros = [rnd.randint(1, 10) for _ in range(7)]
print(numeros)
print(mean(numeros), median(numeros))
print(PI)


## 3. ¿Dónde busca Python los módulos? (`sys.path`)

Python busca módulos en este orden (simplificado):

1. Carpeta actual donde se ejecuta el script / notebook.
2. Rutas del entorno (librería estándar).
3. `site-packages` (paquetes instalados con `pip`).

Puedes ver la lista completa en `sys.path`. Si tu módulo no está en alguna de
esas rutas, tendrás `ModuleNotFoundError`.

⚠️ **Error común**: ejecutar el notebook desde una carpeta distinta a donde
está tu módulo y pensar que Python “lo encontrará solo”.


In [ ]:
import sys

for i, ruta in enumerate(sys.path[:5], start=1):
    print(i, ruta)


## 4. Crear y usar tu propio módulo

Para crear un módulo, basta con un archivo `.py` en la misma carpeta.
Aquí generamos uno de forma programática para el ejemplo.


In [ ]:
from pathlib import Path

Path("mi_modulo.py").write_text(
    "PI = 3.1416
"
    "def area_circulo(r):
"
    "    return PI * r**2
"
    "
"
    "def saludar(nombre='mundo'):
"
    "    return f'Hola, {nombre}'
",
    encoding="utf-8"
)

import mi_modulo

print(mi_modulo.PI)
print(mi_modulo.area_circulo(3))
print(mi_modulo.saludar("Ana"))


## 5. `__name__` y bloques ejecutables

Cuando un archivo se **importa**, su variable `__name__` no es `"__main__"`.
Cuando se **ejecuta directamente**, `__name__` sí es `"__main__"`.

Esto permite incluir pruebas o ejemplos que solo corran si el archivo se ejecuta
como script, pero no cuando se importa.


In [ ]:
from pathlib import Path
import subprocess
import sys

Path("herramientas.py").write_text(
    "def cuadrado(x):
"
    "    return x ** 2
"
    "
"
    "def main():
"
    "    print('cuadrado(5) =', cuadrado(5))
"
    "
"
    "if __name__ == '__main__':
"
    "    main()
",
    encoding="utf-8"
)

import herramientas
print(herramientas.cuadrado(3))

salida = subprocess.check_output([sys.executable, "herramientas.py"], text=True)
print(salida.strip())


## 6. Caché de importación y recarga (`importlib.reload`)

Python **no vuelve a leer** un módulo si ya fue importado en esta sesión.
Si cambias el archivo, necesitas recargarlo manualmente con `importlib.reload`.

⚠️ **Error común**: editar un módulo y no ver cambios porque sigue en caché.


In [ ]:
from pathlib import Path
import importlib

Path("mod_cache.py").write_text("valor = 1
", encoding="utf-8")
import mod_cache
print(mod_cache.valor)

Path("mod_cache.py").write_text("valor = 2
", encoding="utf-8")
importlib.reload(mod_cache)
print(mod_cache.valor)


## 7. Paquetes: organizar módulos en carpetas

Un **paquete** es una carpeta que contiene módulos y un archivo `__init__.py`.
Ese archivo se ejecuta cuando importas el paquete y sirve para exponer una API
más limpia.

Estructura típica:

```
tienda/
  __init__.py
  productos.py
  ventas.py
```

⚠️ **Error común**: olvidar `__init__.py` (en cursos básicos es recomendable
incluirlo siempre para evitar confusiones).


In [ ]:
from pathlib import Path

paquete = Path("tienda")
paquete.mkdir(exist_ok=True)

(paquete / "__init__.py").write_text(
    "from .productos import catalogo, precio
"
    "from .ventas import total
"
    "__all__ = ['catalogo', 'precio', 'total']
",
    encoding="utf-8"
)

(paquete / "productos.py").write_text(
    "catalogo = {'manzana': 12, 'pera': 15, 'naranja': 10}
"
    "
"
    "def precio(nombre):
"
    "    return catalogo.get(nombre)
",
    encoding="utf-8"
)

(paquete / "ventas.py").write_text(
    "from .productos import precio
"
    "
"
    "def total(carrito):
"
    "    return sum(precio(nombre) * cantidad for nombre, cantidad in carrito.items())
",
    encoding="utf-8"
)

from tienda import catalogo, precio, total

print(catalogo)
print(precio("pera"))
print(total({"manzana": 2, "naranja": 3}))


## 8. Importaciones absolutas vs relativas

Dentro de un paquete puedes importar de dos maneras:

- **Absoluta**: desde la raíz del paquete.
- **Relativa**: usando puntos (`.`) para moverte dentro del paquete.

Ejemplo dentro de `tienda/ventas.py`:

```python
from .productos import precio      # relativa
# from tienda.productos import precio  # absoluta
```

⚠️ **Error común**: ejecutar un módulo del paquete “a mano” y obtener
`ImportError: attempted relative import with no known parent package`.
Solución: ejecutar con `python -m tienda.ventas` o usar importaciones absolutas.


## 9. Errores comunes y cómo evitarlos

- ⚠️ `ModuleNotFoundError`: revisa la carpeta actual y `sys.path`.
- ⚠️ “Sombra” de módulos estándar: no nombres archivos como `random.py`.
- ⚠️ Cambios no reflejados: usa `importlib.reload`.
- ⚠️ Importaciones circulares: divide responsabilidades o mueve imports dentro de funciones.
- ⚠️ `from modulo import *`: evita contaminar el espacio de nombres.
- ⚠️ Importaciones relativas al ejecutar módulos: usa `python -m paquete.modulo`.


## 10. Buenas prácticas ✅

1. ✅ Usa nombres de módulos en minúsculas y sin espacios.
2. ✅ Importa solo lo necesario y en un bloque al inicio del archivo.
3. ✅ Evita `import *` y alias poco claros.
4. ✅ Agrupa imports: estándar, terceros, locales.
5. ✅ Expón una API clara en `__init__.py` cuando uses paquetes.
6. ✅ Agrega pruebas o ejemplos con `if __name__ == "__main__":`.


## 11. Ejercicios prácticos

Intenta resolverlos primero y luego compara con la solución.


### Ejercicio 1: Convertidor de temperaturas
**Tarea**: crea `convertidor.py` con `celsius_a_fahrenheit` y `fahrenheit_a_celsius`.
Importa el módulo con un alias y prueba ambas funciones.


In [ ]:
# Tu código aquí:
# ...

# SOLUCIÓN:
from pathlib import Path

Path("convertidor.py").write_text(
    "def celsius_a_fahrenheit(c):
"
    "    return (c * 9/5) + 32
"
    "
"
    "def fahrenheit_a_celsius(f):
"
    "    return (f - 32) * 5/9
",
    encoding="utf-8"
)

import convertidor as conv
print(conv.celsius_a_fahrenheit(0))
print(round(conv.fahrenheit_a_celsius(212), 2))


### Ejercicio 2: Paquete `utilidades`
**Tarea**: crea un paquete con `texto.py` y `numeros.py`. Expón las funciones
en `__init__.py` para importarlas de forma directa.


In [ ]:
# Tu código aquí:
# ...

# SOLUCIÓN:
from pathlib import Path

pkg = Path("utilidades")
pkg.mkdir(exist_ok=True)

(pkg / "__init__.py").write_text(
    "from .texto import limpiar
"
    "from .numeros import es_par, promedio
"
    "__all__ = ['limpiar', 'es_par', 'promedio']
",
    encoding="utf-8"
)

(pkg / "texto.py").write_text(
    "def limpiar(s):
"
    "    return ' '.join(s.strip().split())
",
    encoding="utf-8"
)

(pkg / "numeros.py").write_text(
    "def es_par(n):
"
    "    return n % 2 == 0
"
    "
"
    "def promedio(nums):
"
    "    return sum(nums) / len(nums)
",
    encoding="utf-8"
)

from utilidades import limpiar, es_par, promedio
print(limpiar("  hola   mundo  "))
print(es_par(10), es_par(7))
print(promedio([1, 2, 3, 4]))


### Ejercicio 3: Estadística rápida
**Tarea**: genera 12 números aleatorios y calcula promedio y mediana.


In [ ]:
# Tu código aquí:
# ...

# SOLUCIÓN:
import random
from statistics import mean, median

nums = [random.randint(1, 100) for _ in range(12)]
print(nums)
print(mean(nums), median(nums))


### Ejercicio 4: Módulo en carpeta externa
**Tarea**: crea un módulo dentro de una carpeta `extras/` y agrégala a `sys.path`
para poder importarlo.


In [ ]:
# Tu código aquí:
# ...

# SOLUCIÓN:
from pathlib import Path
import sys

extras = Path("extras")
extras.mkdir(exist_ok=True)

(extras / "saludos.py").write_text(
    "def hola(nombre):
"
    "    return f'Hola, {nombre}'
",
    encoding="utf-8"
)

sys.path.append(str(extras.resolve()))
import saludos
print(saludos.hola("Mina"))


### Ejercicio 5: Bloque principal
**Tarea**: crea un módulo con `main()` y prueba que solo se ejecute al correrlo
como script.


In [ ]:
# Tu código aquí:
# ...

# SOLUCIÓN:
from pathlib import Path
import subprocess
import sys

Path("demo_main.py").write_text(
    "def main():
"
    "    print('Ejecutando como script')
"
    "
"
    "if __name__ == '__main__':
"
    "    main()
",
    encoding="utf-8"
)

import demo_main  # no imprime nada
salida = subprocess.check_output([sys.executable, "demo_main.py"], text=True)
print(salida.strip())


## 12. Resumen de Conceptos Clave

| Concepto | Qué es | Ejemplo |
|----------|--------|---------|
| Módulo | Archivo `.py` con código reutilizable | `mi_modulo.py` |
| Paquete | Carpeta con módulos | `tienda/` |
| `__init__.py` | Inicializa y expone API | `from .x import y` |
| `import` | Importa módulo completo | `import math` |
| `from ... import ...` | Importa nombres específicos | `from math import pi` |
| `as` | Alias para acortar | `import random as rnd` |
| `sys.path` | Rutas de búsqueda | `sys.path.append(...)` |
| `__name__` | Identidad del módulo | `if __name__ == "__main__":` |
| `importlib.reload` | Recarga módulo | `importlib.reload(mod)` |
